In [ ]:
from src.material.material.phong_material import *
from src import World
from src import Sphere
from src import PointLight
from src import Resolution
from src import Camera
from src import Vertex, Vector
from src import Scene
from src.render.loops import *
from src import Plane
from src.render.loops.progress import ProgressDisplay, PreviewConfig
from src.render.render_config import RenderConfig
from src import ipynb_display_images
from src.shading import BlinnPhongShader, NormalShader, DotProductShader, DiffShader
from src.shading import HashMethod
from src.scene import Light
from src.shading.shading_model import ShadingModel
from src import Color
from src.geometry.hit_point import HitPoint

In [ ]:
glossy_brown = PhongMaterial(
    name="glossy_red",
    base_color=Color.custom_rgb(255, 100, 100),
    spec_color=Color.custom_rgb(255, 255, 255),
    shininess=50,
    ior=1.5,
    reflectivity=0.4,
    transparency=0.0
)
world = World()
world.add(
    Plane(point=Vertex(0, -1, 0), normal=Vector(0, 1, 0), material=glossy_brown),
    Sphere(center=Vertex(0, 0.2, -5), radius=1.0, material=glossy_brown)
)
point_light = PointLight(position=Vertex(5, 5, 0), intensity=1000.0, falloff=0.01)
camera = Camera(fov=40, aspect_ratio=Resolution.R360p.aspect_ratio, origin=Vertex(0, 0, 0), direction=Vector(0, 0, -1))
scene = Scene(camera=camera, world=world, lights=[point_light], skybox_path=None)

scene.validate()


In [ ]:
render_config = RenderConfig(
    resolution=Resolution.R480p,
    samples_per_pixel=8,
    max_depth=1,
)

preview_config = PreviewConfig(
    progress_display=ProgressDisplay.TQDM_IMAGE_PREVIEW,
)

# Render with different shaders

## Blinn-Phong Shader
- Uses the Blinn-Phong shading model for rendering

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model= BlinnPhongShader(),
)

path = ray_tracer.render("images/shaders_blinn_phong.png")
ipynb_display_images(path)

## Normal Shader
- Shows the normal vectors as colors

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=NormalShader(),
)

path = ray_tracer.render("images/shaders_normal_shader.png")
ipynb_display_images(path)

# Dot Product Shader (Light Direction and Surface Normal)
- Visualizes the dot product between light direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DotProductShader(use_light=True)
)

path = ray_tracer.render("images/shaders_dot_product_shader_light.png")
ipynb_display_images(path)

# Dot Product Shader (Light Direction and View Direction)
- Visualizes the dot product between view direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DotProductShader(use_light=False)
)

path = ray_tracer.render("images/shaders_dot_product_shader_view.png")
ipynb_display_images(path)

# Diff Shader
- show the difference between light direction and surface normal

In [ ]:
ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DiffShader(a=DotProductShader(use_light=True), b=NormalShader(), hash_method=HashMethod.HALF_IMAGE)
)

path = ray_tracer.render("images/shaders_diff_shader.png")
ipynb_display_images(path)

ray_tracer = MultiProcessRowRenderLoop(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=DiffShader(a=DotProductShader(use_light=True), b=NormalShader(), hash_method=HashMethod.STRIPES)
)

path = ray_tracer.render("images/shaders_diff_shader.png")
ipynb_display_images(path)

# Let's create some custom shader
- For demonstration, we will create a shader that colors based on the position of the hit point

In [ ]:
class MyShader(ShadingModel):
    def shade(self, hit: HitPoint, world: World, light: Light, view_dir: Vector, light_direction=None) -> Color:
        return Color.custom_rgb(
            int((hit.point.x % 1) * 255),
            int((hit.point.y % 1) * 255),
            int((hit.point.z % 1) * 255)
        )

    def shade_multiple_lights(self, hit: HitPoint, world: World, lights: list[Light], view_dir: Vector) -> Color:
        for light in lights:
            return self.shade(hit, world, light, view_dir)

ray_tracer = LinearRayCaster(
    scene=scene,
    render_config=render_config,
    preview_config=preview_config,
    shading_model=MyShader()
)

path = ray_tracer.render("images/shaders_custom_shader.png")